# --- PASO 0: C4RG4 DE D4TOS  ----
B4se de d4tos s4c4d4s de l4 unific4ción de los 12 fomrul4rios. 

### df_resultados_dict
L4 b4se de d4tos de los result4dos contiene l4s column4s:
- ID_sujeto (Suj_001...)
- origen_form (Respuestas de formulario 1...)
- identidad (ABC...)
- bloque (Bloque_1...)
- dilema (Bloque_CON...)
- categoria (1...)
- respuesta (Opción 2...)
contiene 21 fil4s por c4d4 sujeto

### df_demograficos_dict
L4 b4se de d4tos de los cuestion4rios contiene l4s column4s:
- ID_Sujeto (Suj_001...)
- Origen_Form (Respuestas de formulario 1...)
- expectativa_sin, expectativa_grande, expectativa_pequeña (pregunt4s de expect4tiv4)
- ndc_1, ndc_2,	ndc_3,	ndc_4,	ndc_5,	ndc_6 (need for cognition)
- sdo_1, sdo_2,	sdo_3,	sdo_4,	sdo_5,	sdo_6,	sdo_7, sdo_8, sdo_9, sdo_10 (soci4l domin4nce orient4tion)
- Genero 
- politica 
- nivel_se

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import chi2_contingency

In [2]:
# 1. Cargar las bases de datos
df_res_dict = pd.read_csv('../data/raw/Base_res_dict.csv')
df_dem_dict = pd.read_csv('../data/raw/Base_Dem_dict.csv')
df_res_coop = pd.read_csv('../data/raw/Base_res_coop.csv')
df_info_bloques = pd.read_csv('../data/raw/Inform4ción blo#ues.csv')

# 2. Pre-procesamiento de Info Bloques para el merge
# Renombramos 'Nombre' a 'Bloque' para que coincida con df_res_dict
df_info_bloques = df_info_bloques.rename(columns={'Nombre': 'Bloque'})
# Seleccionamos variables de interés (Gap)
# Usamos 'Diferencia desigualdad final entre opciones' como el costo de revertir
cols_interes = ['Bloque', 'Diferencia desigualdad final entre opciones']
df_info_bloques_sel = df_info_bloques[cols_interes]

print("Datos cargados correctamente.")
print(f"Sujetos totales: {df_res_dict['ID_Sujeto'].nunique()}")

Datos cargados correctamente.
Sujetos totales: 162


## --- PASO 1: LIMPIEZA  ---
-Se elimin4n los sujetos #ue f4ll4ron l4s pregunt4s de 4tención (con f4ll4r 1 y4 se elimin4)
Recibe:
- df_demograficos_dict
- df_resultados_dict

162 sujetos

- df_resultados_coop
- df_demograficos_coop

46 sujetos


Devuelve:
- df_res_dict_clean
- df_demo_dict_clean

121 sujetos

- df_res_coop_clean
- df_demo_coop_clean

42 sujetos

In [3]:
# 1. Identificar fallos de atención
df_att = df_res_dict[df_res_dict['Bloque'] == 'Bloque_0']
failed_ids = df_att[df_att['Respuesta'] != 'Opción 3']['ID_Sujeto'].unique()

# 2. Filtrar Dataframes
df_res_clean = df_res_dict[~df_res_dict['ID_Sujeto'].isin(failed_ids)].copy()
df_dem_clean = df_dem_dict[~df_dem_dict['ID_Sujeto'].isin(failed_ids)].copy()
df_coop_clean = df_res_coop[~df_res_coop['ID_Sujeto'].isin(failed_ids)].copy()

# 3. Eliminar filas que no son de interés para el análisis (Bloque_0, Dilema 'Atencion')
df_res_clean = df_res_clean[df_res_clean['Bloque'] != 'Bloque_0']
df_res_clean = df_res_clean[df_res_clean['Dilema'] != 'Atencion']

print(f"Sujetos eliminados por atención: {len(failed_ids)}")
print(f"Muestra final limpia: {df_res_clean['ID_Sujeto'].nunique()}")


Sujetos eliminados por atención: 41
Muestra final limpia: 121


## --- PASO 2: C4mbio de v4ri4bles  ---
- Se cre4 l4 v4ri4ble "Mantiene" (v4le 1 se responde opcion 1)
- Se c4mbi4 l4 v4ri4ble expect4tiv4 (v4le 1, 0, -1; h4y efecto positivo en opc1, no lo h4y, h4y efecto positivo en opc2)
- Se cre4 Scores Psicométricos (SDO y NDC)
- Se cre4 df_master (Unimos Resultados + Demográficos + Info Bloques )
- se cre4 column4 de expect4tiv4s_num (#ue respondió c4d4 sujeto en c4d4 g4p)
- se c4mbi4 nombre de column4
-     "Diferencia desigualdad final entre opciones" --> Gap_Size

In [4]:
# ==============================================================================
# cre4ción de column4 expect4tiv4 4ctiv4
# ==============================================================================

def asignar_expectativa_segura(row):
    bloque = row['Bloque']
    
    # Bloques Sin Gap
    if bloque in ["Bloque_1", "Bloque_3"]:
        return row.get('expectativa_sin_num', np.nan)
        
    # Bloques Gap Grande
    elif bloque in ["Bloque_5", "Bloque_7"]:
        return row.get('expectativa_grande_num', np.nan)
        
    # Bloques Gap Pequeño
    elif bloque in ["Bloque_9", "Bloque_11"]:
        return row.get('expectativa_pequeña_num', np.nan)
        
    else:
        return np.nan

In [5]:
# 1. Codificar Variable Dependiente: REVERSIÓN DE RANKING
# Opción 2 = Revierte el ranking -> Codificamos como 0 (Reversión)
# Opción 1 = Mantiene el ranking -> Codificamos como 1 (Mantiene)
print("Conteo Original:")
print(df_res_clean['Respuesta'].value_counts())

df_res_clean['Mantiene'] = df_res_clean['Respuesta'].apply(lambda x: 1 if x == 'Opción 1' else 0)

print("\nConteo Nueva Variable:")
print(df_res_clean['Mantiene'].value_counts())

Conteo Original:
Opción 2    1357
Opción 1     821
Name: Respuesta, dtype: int64

Conteo Nueva Variable:
0    1357
1     821
Name: Mantiene, dtype: int64


In [6]:
# 2. C4mbi4mos l4 sección de expect4tiv4s. Respuest4s y titulos p4r4 3ue se4 m4s sencillo
#respuest4s de expect4tiv4s. Definimos el diccionario con el mapeo exacto
mapeo_expectativas = {
    "No espero ningún efecto": 0,
    "La Opción 1 incrementa la probabilidad de cooperación": 1,
    "La opción 1 incrementa la posibilidad de cooperación" : 1,
    "La Opción 2 incrementa la probabilidad de cooperación": -1,
    "La opción 2 incrementa la posibilidad de cooperación": -1
}

cols_expectativas = ['expectativa_sin', 'expectativa_grande', 'expectativa_pequeña']

# Aplicamos el reemplazo
for col in cols_expectativas:
    if col in df_dem_clean.columns:
        df_dem_clean[col] = df_dem_clean[col].replace(mapeo_expectativas)

# CHE#UEO R4PIDO
#--------------------------------------------------------
## 1. list de column4s 4 prob4r
#cols_reales = ['expectativa_sin', 'expectativa_grande', 'expectativa_pequeña']

## 2. Verificamos columna por columna
#for col in cols_reales:
#    if col in df_dem_clean.columns:
#        valores = df_dem_clean[col].unique()
#        print(f"[{col}] Valores únicos: {valores}")
        
        # Alerta rápida si detecta texto
#        if df_dem_clean[col].dtype == 'object':
#             print(f"⚠️ ALERTA: La columna {col} todavía es texto. Revisa espacios o tildes.")
#    else:
#        print(f"❌ ERROR: No encuentro la columna '{col}'. Revisa si escribiste bien el nombre en la base de datos.")
#    print("-" * 30)
#-----------------------------------------------------------------

In [7]:
# 3. Calcular Scores Psicométricos (SDO y NDC)
sdo_cols = [c for c in df_dem_clean.columns if 'sdo_' in c]
ndc_cols = [c for c in df_dem_clean.columns if 'ndc_' in c]

# (Asumimos que no hay ítems reversos o que ya están invertidos. Promediamos).
df_dem_clean['SDO_Score'] = df_dem_clean[sdo_cols].mean(axis=1)
df_dem_clean['NDC_Score'] = df_dem_clean[ndc_cols].mean(axis=1)

In [8]:
# 4. Merge Final (Unir todo en un Master DataFrame)
# Unimos Resultados + Demográficos + Info Bloques 
cols_demograficas = [
    'ID_Sujeto', 
    'SDO_Score', 
    'NDC_Score',
    'expectativa_sin',    
    'expectativa_grande',   
    'expectativa_pequeña',
    'Genero', 
    'politica',
    'nivel_se'
]
df_master = df_res_clean.merge(df_dem_clean[cols_demograficas], on='ID_Sujeto', how='left')
df_master = df_master.merge(df_info_bloques_sel, on='Bloque', how='left')

In [9]:
#df_master.columns.tolist()

In [10]:
# 5. Creo un4 column4 nuev4 sobre el df_m4ster
cols_exp = ['expectativa_sin', 'expectativa_grande', 'expectativa_pequeña']

for col in cols_exp:
    # Creamos las versiones numéricas: ej. 'expectativa_sin_num'
    if col in df_master.columns:
        df_master[f'{col}_num'] = df_master[col].fillna(0)

# Aplicamos la función
df_master['Expectativa_Activa'] = df_master.apply(asignar_expectativa_segura, axis=1)

In [11]:
# 6. Renombrar columna de Gap para facilitar uso
df_master.rename(columns={'Diferencia desigualdad final entre opciones': 'Gap_Size'}, inplace=True)


In [12]:

col_gap = 'Gap_Size' 

# Esto creará 5 columnas nuevas (una por cada tamaño de gap) para cada sujeto
df_gaps = df_master.pivot_table(index='ID_Sujeto', 
                         columns=col_gap, 
                         values='Mantiene', 
                         aggfunc='mean')

# 4. RENOMBRAR LAS NUEVAS COLUMNAS
df_gaps = df_gaps.add_prefix('Promedio_Gap_')

# Verificamos qué se creó
print("Nuevas variables creadas:", df_gaps.columns.tolist())

# 5. INTEGRACIÓN (MERGE) A LA BASE MAESTRA
# Reseteamos el índice para poder usar ID_Sujeto en el merge
df_gaps_reset = df_gaps.reset_index()

# Hacemos el cruce (Left Join)
# Esto pega las 5 columnas nuevas a la derecha de tu tabla gigante
df_master = pd.merge(df_master, df_gaps_reset, on='ID_Sujeto', how='left')

# ==============================================================================
# 6. GUARDAR
# ==============================================================================
print("\n--- Vista Previa (Primer sujeto) ---")
# Mostramos el ID y las nuevas columnas de Gap
cols_to_show = ['ID_Sujeto'] + df_gaps.columns.tolist()
print(df_master[cols_to_show].head(1))


Nuevas variables creadas: ['Promedio_Gap_0', 'Promedio_Gap_1000', 'Promedio_Gap_1200', 'Promedio_Gap_2000', 'Promedio_Gap_2400']

--- Vista Previa (Primer sujeto) ---
  ID_Sujeto  Promedio_Gap_0  Promedio_Gap_1000  Promedio_Gap_1200  \
0   Suj_001        0.666667           0.666667           0.666667   

   Promedio_Gap_2000  Promedio_Gap_2400  
0           0.333333                0.0  


In [13]:
print("Variables creadas: Mantiene, SDO_Score, NDC_Score, Gap_Size, Expectativa_Activa")
print(df_master[['ID_Sujeto', 'Bloque','Dilema', 'Mantiene', 'Gap_Size', 'SDO_Score', 'Expectativa_Activa']].head())

Variables creadas: Mantiene, SDO_Score, NDC_Score, Gap_Size, Expectativa_Activa
  ID_Sujeto    Bloque      Dilema  Mantiene  Gap_Size  SDO_Score  \
0   Suj_001  Bloque_1  Bloque_CON         0         0        3.3   
1   Suj_001  Bloque_5  Bloque_CON         0      2000        3.3   
2   Suj_001  Bloque_7  Bloque_CON         0      2400        3.3   
3   Suj_001  Bloque_3  Bloque_CON         0         0        3.3   
4   Suj_001  Bloque_9  Bloque_CON         1      1000        3.3   

   Expectativa_Activa  
0                   0  
1                  -1  
2                  -1  
3                   0  
4                  -1  


In [14]:
df_wide = df_master.pivot_table(index='ID_Sujeto', 
                         columns='Dilema', 
                         values='Mantiene', 
                         aggfunc='mean')

# 3. CÁLCULO DEL DELTA (VARIABLE DEPENDIENTE)
# Fórmula: Delta = Promedio(CON) - Promedio(SIN)  
# El Cálculo Matemático
df_wide['Promedio_CON'] = df_wide['Bloque_CON']
df_wide['Promedio_SIN'] = df_wide['Bloque_SIN']
df_wide['Promedio_DIST'] = df_wide['Dist']
df_wide['Delta_Mantiene'] = df_wide['Promedio_CON'] - df_wide['Promedio_SIN']
df_wide['Delta_base'] = df_wide['Promedio_SIN'] - df_wide['Promedio_DIST']
df_wide.head()

columnas_finales = ['Promedio_CON', 'Promedio_SIN', 'Promedio_DIST', 'Delta_Mantiene', 'Delta_base']
df_delta = df_wide[columnas_finales]

In [15]:
df_delta.head()
#df_delta.to_csv('../data/processed/df_delta.csv', index=False)

Dilema,Promedio_CON,Promedio_SIN,Promedio_DIST,Delta_Mantiene,Delta_base
ID_Sujeto,,,,,
Suj_001,0.166667,0.666667,0.666667,-0.500000,0.000000
Suj_003,0.333333,0.166667,0.166667,0.166667,0.000000
Suj_004,0.333333,0.166667,0.166667,0.166667,0.000000
Suj_006,0.666667,0.500000,0.333333,0.166667,0.166667
Suj_007,0.500000,0.500000,0.500000,0.000000,0.000000


In [16]:
df_long1 = pd.merge(df_master, df_delta, on='ID_Sujeto', how='left')
df_long1.head()

,ID_Sujeto,Origen_Form,Identidad,Bloque,Dilema,Categoria,Respuesta,Mantiene,SDO_Score,NDC_Score,...,Promedio_Gap_0,Promedio_Gap_1000,Promedio_Gap_1200,Promedio_Gap_2000,Promedio_Gap_2400,Promedio_CON,Promedio_SIN,Promedio_DIST,Delta_Mantiene,Delta_base
0,Suj_001,Respuestas de formulario 1,ABC,Bloque_1,Bloque_CON,1,Opción 2,0,3.3,2.833333,...,0.666667,0.666667,0.666667,0.333333,0.0,0.166667,0.666667,0.666667,-0.5,0.0
1,Suj_001,Respuestas de formulario 1,AEF,Bloque_5,Bloque_CON,2,Opción 2,0,3.3,2.833333,...,0.666667,0.666667,0.666667,0.333333,0.0,0.166667,0.666667,0.666667,-0.5,0.0
2,Suj_001,Respuestas de formulario 1,AJK,Bloque_7,Bloque_CON,3,Opción 2,0,3.3,2.833333,...,0.666667,0.666667,0.666667,0.333333,0.0,0.166667,0.666667,0.666667,-0.5,0.0
3,Suj_001,Respuestas de formulario 1,AGH,Bloque_3,Bloque_CON,4,Opción 2,0,3.3,2.833333,...,0.666667,0.666667,0.666667,0.333333,0.0,0.166667,0.666667,0.666667,-0.5,0.0
4,Suj_001,Respuestas de formulario 1,ALM,Bloque_9,Bloque_CON,5,Opción 1,1,3.3,2.833333,...,0.666667,0.666667,0.666667,0.333333,0.0,0.166667,0.666667,0.666667,-0.5,0.0


In [17]:
df_master.columns.tolist()

['ID_Sujeto',
 'Origen_Form',
 'Identidad',
 'Bloque',
 'Dilema',
 'Categoria',
 'Respuesta',
 'Mantiene',
 'SDO_Score',
 'NDC_Score',
 'expectativa_sin',
 'expectativa_grande',
 'expectativa_pequeña',
 'Genero',
 'politica',
 'nivel_se',
 'Gap_Size',
 'expectativa_sin_num',
 'expectativa_grande_num',
 'expectativa_pequeña_num',
 'Expectativa_Activa',
 'Promedio_Gap_0',
 'Promedio_Gap_1000',
 'Promedio_Gap_1200',
 'Promedio_Gap_2000',
 'Promedio_Gap_2400']

In [18]:
df_long1.columns.tolist()

['ID_Sujeto',
 'Origen_Form',
 'Identidad',
 'Bloque',
 'Dilema',
 'Categoria',
 'Respuesta',
 'Mantiene',
 'SDO_Score',
 'NDC_Score',
 'expectativa_sin',
 'expectativa_grande',
 'expectativa_pequeña',
 'Genero',
 'politica',
 'nivel_se',
 'Gap_Size',
 'expectativa_sin_num',
 'expectativa_grande_num',
 'expectativa_pequeña_num',
 'Expectativa_Activa',
 'Promedio_Gap_0',
 'Promedio_Gap_1000',
 'Promedio_Gap_1200',
 'Promedio_Gap_2000',
 'Promedio_Gap_2400',
 'Promedio_CON',
 'Promedio_SIN',
 'Promedio_DIST',
 'Delta_Mantiene',
 'Delta_base']

In [19]:
# --- PASO EXTRA: FILTRO DE GÉNERO ---

# 1. Definimos las categorías que SÍ queremos conservar
# (Asegúrate de escribirlas tal cual aparecen en tu Excel, ej: 'Mujer', 'Hombre' o 'Femenino', 'Masculino')
generos_validos = ['Mujer', 'Hombre']

# 2. Filtramos la tabla demográfica
df_long = df_long1[df_long1['Genero'].isin(generos_validos)].copy()

# Verificación
print(f"Sujetos eliminados por género: {len(df_long1) - len(df_long)}")
print(f"N Final para análisis: {len(df_long)}")
print("Distribución final:")
print(df_long['Genero'].value_counts())

Sujetos eliminados por género: 54
N Final para análisis: 2142
Distribución final:
Mujer     1458
Hombre     684
Name: Genero, dtype: int64


In [20]:
df_long.to_csv('../data/processed/df_long.csv', index=False)
print("\n✅ ¡Listo! Archivo 'Base_Maestra_Completa_ConDeltas.csv' generado correctamente.")


✅ ¡Listo! Archivo 'Base_Maestra_Completa_ConDeltas.csv' generado correctamente.
